In [1]:
"""
Created on Wed Aug  4 17:50:06 2021

@author: amol
"""

# set TF GPU memory growth so that it doesn't hog everything at once
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import os
import numpy as np
from main import *
debug_mode = True  # faster experiments for debugging

# util for getting objects' fields' names
field_names = lambda x: list(vars(x).keys())

In [2]:
#Prepare the DataFrame that will be used downstream
dp = DataPreparer()
dp.createPCAs()
dp.sparsePCAs()
dp.zmixOrthogonalPCAs()
df = dp.getDataframe()

# currently passing dp eventually we want to abstract all the constants into 1 class
dm = DataManager(df, dp)

In [3]:
"""
bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
dm.createTrainTestData(experimentSettings['dataSetMethod'], experimentSettings['noOfCpv'],
                       experimentSettings['ipscaler'], experimentSettings['opscaler'])
dm.input_data_cols
"""

"\nbestModel, experimentSettings = exprExec.modelFactory.openBestModel()\ndm.createTrainTestData(experimentSettings['dataSetMethod'], experimentSettings['noOfCpv'],\n                       experimentSettings['ipscaler'], experimentSettings['opscaler'])\ndm.input_data_cols\n"

In [4]:
'''
Run the Model Experiments
'''
model = 'PCDNN_V2'
assert model in ['PCDNN_V2', 'PCDNN_V1', 'SIMPLE_DNN', 'GP'] # valid possible models
exprExec=run_model_experiments(dm, models=model, debug_mode=False)

/home/dwyerdei/rom_project2/src/experiment_executor/pcdnn_v2_experiment_executor.py:33: UserWarning: manually overriding n models to 1! change this!
  warnings.warn('manually overriding n models to 1! change this!')


Parent DNNModelFactory Instantiated
=================== randomequaltraintestsplit ===================
------------------ AllSpeciesAndZmix ------------------
--------------------self.build_and_compile_pcdnn_v2_model----------------------
53 4 Y Y Y


KeyboardInterrupt: 

In [3]:
"""
print('Organizing results...')
os.system('rm -r data_results data_results.zip')
os.system('mkdir data_results')
os.system('mv PCA_data.csv ./data_results/.')
os.system('mv PCDNNV2_Experiment_Results.csv ./data_results/.')
os.system('cp -r ./models/best_models data_results/.')
os.system('zip -r data_results.zip data_results; rm -r data_results')
print('Done!')
"""

"\nprint('Organizing results...')\nos.system('rm -r data_results data_results.zip')\nos.system('mkdir data_results')\nos.system('mv PCA_data.csv ./data_results/.')\nos.system('mv PCDNNV2_Experiment_Results.csv ./data_results/.')\nos.system('cp -r ./models/best_models data_results/.')\nos.system('zip -r data_results.zip data_results; rm -r data_results')\nprint('Done!')\n"

In [3]:
""" prepare SimpleDNN for loading (from prior experiments) """

exprExec = DNNExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(SimpleDNNModelFactory())

Parent DNNModelFactory Instantiated


In [4]:
""" prepare PCDNNV1 for loading (from prior experiments) """

exprExec = PCDNNV1ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV1ModelFactory())

Parent DNNModelFactory Instantiated


In [5]:
""" prepare PCDNNV2 for loading (from prior experiments) """

exprExec = PCDNNV2ExperimentExecutor()
exprExec.debug_mode = debug_mode

exprExec.setModelFactory(PCDNNV2ModelFactory())

Parent DNNModelFactory Instantiated


/home/dwyerdei/rom_project2/src/experiment_executor/pcdnn_v2_experiment_executor.py:33: UserWarning: manually overriding n models to 1! change this!
  warnings.warn('manually overriding n models to 1! change this!')


In [ ]:
# fix seeds
import random
import numpy as np
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

dataType = 'randomequaltraintestsplit' #'frameworkincludedtrainexcludedtest'
inputType = 'AllSpeciesAndZmix'
dataSetMethod = f'{inputType}_{dataType}'
opscaler = "MinMaxScaler"# 'PositiveLogNormal'
ZmixPresent = 'Y'
concatenateZmix = 'Y'
kernel_constraint = 'N'
kernel_regularizer = 'N'
activity_regularizer = 'N'
noOfCpv = 4
noOfNeurons = 53

exprExec.modelFactory.loss='mae'#tf.keras.losses.MeanAbsoluteError()#'mse'
exprExec.modelFactory.activation_func='relu'
exprExec.modelFactory.dropout_rate=0.5
exprExec.debug_mode = False
exprExec.epochs_override = 10
exprExec.batch_size = 32
exprExec.n_models_override = 1

# initialize experiment executor...
exprExec.dm = dm
exprExec.df_experimentTracker = pd.DataFrame()
exprExec.modelType = 'PCDNNV2'

history = exprExec.executeSingleExperiment(noOfNeurons,dataSetMethod,dataType,inputType,ZmixPresent=ZmixPresent,
                                           noOfCpv=noOfCpv,concatenateZmix=concatenateZmix,kernel_constraint=kernel_constraint,
                                           kernel_regularizer=kernel_regularizer,activity_regularizer=activity_regularizer,
                                           opscaler=opscaler)

/home/dwyerdei/rom_project2/src/models/pcdnnv2_model_factory.py:154: UserWarning: manually overriding loss, fixme!
  warnings.warn('manually overriding loss, fixme!')


--------------------self.build_and_compile_pcdnn_v2_model----------------------
53 4 N N N
using loss: mae
> /home/dwyerdei/rom_project2/src/models/pcdnnv2_model_factory.py(156)build_and_compile_model()
    154         warnings.warn('manually overriding loss, fixme!')
    155         import pdb; pdb.set_trace()
--> 156         model.compile(loss=self.loss,optimizer=opt, metrics=[self.loss])#, 'mse',  exp_mse_mag, exp_mae_mag, exp_R2, R2])
    157 
    158         self.model = model

ipdb> c
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 species_input (InputLayer)     [(None, 53)]         0           []                               
                                                                                                  
 zmix (InputLayer)              [(None, 1)]          0           []                        

ipdb> p self.model.save('weird_model.h5')
None


In [7]:
print(exprExec.modelFactory.model.compiled_loss)
print(exprExec.modelFactory.model.compiled_metrics)

loss = 658787795.4682382

def print_scientific_notation(number):
    power = int(np.log(number)/np.log(10))
    print(f"Scientific Notation: {(loss/10**power)}*10^{power}")
print_scientific_notation(loss)

Scientific Notation: 6.587877954682383*10^8


In [9]:
?exprExec.executeSingleExperiment

In [20]:
field_names(exprExec.modelFactory.model)

['_self_setattr_tracking',
 '_is_model_for_instrumentation',
 '_instrumented_keras_api',
 '_instrumented_keras_layer_class',
 '_instrumented_keras_model_class',
 '_trainable',
 '_stateful',
 'built',
 '_input_spec',
 '_build_input_shape',
 '_saved_model_inputs_spec',
 '_saved_model_arg_spec',
 '_supports_masking',
 '_name',
 '_activity_regularizer',
 '_trainable_weights',
 '_non_trainable_weights',
 '_updates',
 '_thread_local',
 '_callable_losses',
 '_losses',
 '_metrics',
 '_metrics_lock',
 '_dtype_policy',
 '_compute_dtype_object',
 '_autocast',
 '_self_tracked_trackables',
 '_inbound_nodes_value',
 '_outbound_nodes_value',
 '_expects_training_arg',
 '_default_training_arg',
 '_expects_mask_arg',
 '_dynamic',
 '_initial_weights',
 '_auto_track_sub_layers',
 '_preserve_input_structure_in_config',
 '_outer_name_scope',
 '_is_graph_network',
 'inputs',
 'outputs',
 'input_names',
 'output_names',
 '_compute_output_and_mask_jointly',
 '_distribution_strategy',
 '_cluster_coordinator',
 

## Debugging Code:

In [ ]:
dm.createTrainTestData(dataSetMethod, noOfCpv, opscaler, opscaler)
def getData():
    control_keys = ['X_train', 'X_test', 'Y_train', 'Y_test', 'Zmix_train', 'Zmix_test']
    X_train, X_test, Y_train, Y_test, rom_train, rom_test, Zmix_train, Zmix_test = dm.getTrainTestData()
    return {k:v for k,v in locals().items() if k in control_keys}

control_dict = getData()
print(control_dict.keys())

In [ ]:
locals().update(control_dict)
to_series = lambda x: map(pd.DataFrame, x)
train_df = pd.concat(to_series([X_train, Y_train, Zmix_train]), axis=1)
test_df = pd.concat(to_series([X_test, Y_test, Zmix_test]), axis=1)
show_shape = lambda x: print(f'{x} shape: {globals()[x].shape}')
for name in control_dict.keys():
    show_shape(name)

In [ ]:
pd.Series(Zmix_train).describe()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
print(field_names(dm))
dm.input_data_cols

In [ ]:
"""
import seaborn as sns
#df.describe()
#df['souener'].describe()
sns.pairplot(df[df.columns[:-5]])
"""

## Debugging Code End

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.plot(np.maximum(history.history['val_R2'][30:], 0))
plt.title('val_R^2 vs epochs')
plt.ylabel('percent')
plt.xlabel('epochs')
plt.show()

In [ ]:
import numpy as np
print(np.log(2189882659.968396)/np.log(10))
print(np.log(2099652636.0322459)/np.log(10))
print(np.log(5608740345.916335)/np.log(10))
#vars(exprExec.dm.outputScaler.log_col_transformers[0])
#exprExec.df_err

print(np.exp(np.sqrt(0.0370)))


In [ ]:
good_model = exprExec.model
print(good_model)

In [ ]:
import keras
help(keras.models.Model.fit)

In [ ]:
import model_analyzer.model_analysis as model_analysis
import importlib; importlib.reload(model_analysis)

bestModel, experimentSettings = exprExec.modelFactory.openBestModel()
dm.createTrainTestData(experimentSettings['dataSetMethod'],experimentSettings['noOfCpv'], experimentSettings['ipscaler'], experimentSettings['opscaler'])

print(f'\nexperimentSettings: {experimentSettings}')
print(f'\nbestModel.input_shape: {bestModel.input_shape}')
inspector = model_analysis.ModelInspector(exprExec.modelFactory, dm)

In [ ]:
n_repeats = 5 if debug_mode else 20
inspector.plot_permutation_feature_importance(n_repeats=n_repeats)

In [ ]:
inspector.plot_partial_dependence()

### Print & Record Linear Embeddings Output

In [ ]:
import numpy as np

def inspect_PCA(X):
    PCA_dict = {}
    PCA_dict['X'] = X
    PCA_dict['Var'] = PCA_dict['X'].var(axis=0)
    
    # sort by explained variance
    sort_idx = np.argsort(PCA_dict['Var'])[::-1]
    PCA_dict['X'] = PCA_dict['X'][:, sort_idx]
    PCA_dict['Var'] = PCA_dict['Var'][sort_idx]
    PCA_dict['CumVar'] = np.add.accumulate(PCA_dict['Var'])
    return PCA_dict


X, Y, rom, zmix = dm.getAllData()
FullDataset = inspect_PCA(X)
try:    
    linearAutoEncoder = exprExec.modelFactory.getLinearEncoder()
    X = linearAutoEncoder.predict(FullDataset['X'])
    DNN_PCA = inspect_PCA(X)
except KeyError:
    print('No Linear Auto Encoder!')

In [ ]:
#dm.createDataset
import matplotlib.pyplot as plt
import copy
dm_PurePCA = copy.deepcopy(dm)
dm_PurePCA.createTrainTestData(dataSetMethod='PurePCA_randomequalflamesplit',
                               numCpvComponents=2, ipscaler=None, opscaler=None)

X,Y,rom,zmix = dm_PurePCA.getAllData()
PurePCA = inspect_PCA(X)
print(PurePCA['Var'])
print(DNN_PCA['Var'])

plt.plot(PurePCA['CumVar']/FullDataset['Var'].sum(), color='r')
plt.plot(DNN_PCA['CumVar']/FullDataset['Var'].sum(), color='b')
plt.title('PCA Total Variance Comparison')